# Character level language model - Dinosaurus Island

This notebook, adapted from Deeplearning.ai's Deep Learning course, 
builds a character-level language model to give names to dinosaurs.

<!-- <table>
<td>
<img src="images/dino.jpg" style="width:250;height:300px;">

</td>
</table> -->

A list of all the dinosaur names are compiled into this [dataset](dinos.txt). To create new dinosaur names, the algorithm will learn the different name patterns, and randomly generate new names.

**Objectives:**

* Store text data for processing using an RNN 
* Build a character-level text generation model using an RNN
* Sample novel sequences in an RNN
* Explain the vanishing/exploding gradient problem in RNNs
* Apply gradient clipping as a solution for exploding gradients

In [1]:
# uncomment the following line to install the packages.
# !pip install numpy

In [2]:
import numpy as np
# Functions such as `rnn_forward` and `rnn_backward` are provided in `rnn_utils`
from utils import *
import random
import pprint
import copy

### Dataset and Preprocessing

Run the following cell to read the dataset of dinosaur names, create a list of unique characters (such as a-z), and compute the dataset and vocabulary size. 

In [3]:
data = open('dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 19909 total characters and 27 unique characters in your data.



* The characters are a-z (26 characters) plus the "\n" (or newline character).
* The newline character "\n" plays a role similar to the `<EOS>` (or "End of sentence") token discussed in lecture.  
    - Here, "\n" indicates the end of the dinosaur name rather than the end of a sentence. 
* `char_to_ix`: In the cell below, we'll create a Python dictionary (i.e., a hash table) to map each character to an index from 0-26.
* `ix_to_char`: Then, we'll create a second Python dictionary that maps each index back to the corresponding character. 
    -  This will clarify which index in the probability distribution output of the softmax layer corresponds to each character.

In [4]:
chars = sorted(chars)
print(chars)

['\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(ix_to_char)

{   0: '\n',
    1: 'a',
    2: 'b',
    3: 'c',
    4: 'd',
    5: 'e',
    6: 'f',
    7: 'g',
    8: 'h',
    9: 'i',
    10: 'j',
    11: 'k',
    12: 'l',
    13: 'm',
    14: 'n',
    15: 'o',
    16: 'p',
    17: 'q',
    18: 'r',
    19: 's',
    20: 't',
    21: 'u',
    22: 'v',
    23: 'w',
    24: 'x',
    25: 'y',
    26: 'z'}


### Overview of the Model

The model has the following structure:

- **Initialize Parameters**: Set up initial values for model parameters.
- **Run the Optimization Loop**:
    - **Forward Propagation**: Compute the loss function.
    - **Backward Propagation**: Calculate the gradients with respect to the loss function.
    - **Gradient Clipping**: Prevent exploding gradients by clipping them.
    - **Parameter Update**: Use the gradients to update parameters via the gradient descent rule.
- **Return Learned Parameters**: Output the optimized parameters.

<img src="images/rnn.png" style="width:450;height:300px;">
<caption><center><font><b>Figure 1</b>: Recurrent Neural Network, similar to the model built in the previous notebook "Building a Recurrent Neural Network - Step by Step."</center></caption>

- At each time step, the RNN predicts the next character based on previous characters.
- $\mathbf{X} = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ represents a sequence of characters from the training set.
- $\mathbf{Y} = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$ is the same sequence, but shifted one character forward.
- At each time step $t$, $y^{\langle t \rangle}$ equals $x^{\langle t+1 \rangle}$. Therefore, the prediction at time $t$ corresponds to the input at time $t + 1$.


## Building Blocks of the Model

In this part, we will build two important blocks of the overall model:

1. Gradient clipping: to avoid exploding gradients
2. Sampling: a technique used to generate characters

### Clipping the Gradients in the Optimization Loop

This section focuses on implementing the `clip` function, which will be utilized within the optimization loop.

#### Exploding Gradients

- **Exploding Gradients**: When gradients become excessively large, they are referred to as "exploding gradients." This issue complicates the training process because large updates can cause the model parameters to "overshoot" optimal values during backpropagation.

Typically, the overall loop structure includes:
- Forward pass
- Cost computation
- Backward pass
- Parameter update

Before updating the parameters, gradient clipping is performed to ensure that gradients do not become excessively large.

#### Gradient Clipping

In the following exercise, the `clip` function will be implemented to handle gradients in a dictionary, returning a clipped version if necessary.

- **Gradient Clipping Methods**: Various techniques can be used to clip gradients. Here, a simple element-wise clipping approach is applied.
- **Element-Wise Clipping**: Each element of the gradient vector is constrained within a range [-N, N]. 
    - For example, with $ N = 10 $:
        - The clipping range is [-10, 10].
        - Any gradient component exceeding 10 is set to 10.
        - Any gradient component below -10 is set to -10.
        - Gradient components between -10 and 10 retain their original values.

<img src="images/clip.png" style="width:400;height:150px;">
<caption><center><b>Figure 2</b>: Visualization of gradient descent with and without gradient clipping, demonstrating how gradient clipping addresses "exploding gradient" issues.</center></caption>


### clip

The `clip` function returns the clipped gradients from the dictionary `gradients`.

- **Function Purpose**: The function takes a maximum threshold and applies clipping to the gradients in the dictionary.
- **Usage of `numpy.clip`**: For more details, refer to [numpy.clip](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.clip.html).
    - Utilize the `out` parameter to perform in-place updates.
    - If the `out` argument is not used, the clipped gradients are computed but not applied to the gradient variables (`dWax`, `dWaa`, `dWya`, `db`, `dby`).

The `clip` function ensures that the gradients do not exceed the specified threshold, maintaining stability during the optimization process.


In [6]:
def clip(gradients, maxValue):
    '''
    Clips the gradients' values between minimum and maximum.
    
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- everything above this number is set to this number, and everything less than -maxValue is set to -maxValue
    
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    '''
    gradients = copy.deepcopy(gradients)
    
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    # Clip to mitigate exploding gradients, loop over [dWax, dWaa, dWya, db, dby].
    for gradient in gradients:
        np.clip(gradients[gradient], -maxValue, maxValue, out = gradients[gradient])
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

In [7]:
# Test with a max value of 10
def clip_test(target, mValue):
    print(f"\nGradients for mValue={mValue}")
    np.random.seed(3)
    dWax = np.random.randn(5, 3) * 10
    dWaa = np.random.randn(5, 5) * 10
    dWya = np.random.randn(2, 5) * 10
    db = np.random.randn(5, 1) * 10
    dby = np.random.randn(2, 1) * 10
    gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}

    gradients2 = target(gradients, mValue)
    print("gradients[\"dWaa\"][1][2] =", gradients2["dWaa"][1][2])
    print("gradients[\"dWax\"][3][1] =", gradients2["dWax"][3][1])
    print("gradients[\"dWya\"][1][2] =", gradients2["dWya"][1][2])
    print("gradients[\"db\"][4] =", gradients2["db"][4])
    print("gradients[\"dby\"][1] =", gradients2["dby"][1])
    
    for grad in gradients2.keys():
        valuei = gradients[grad]
        valuef = gradients2[grad]
        mink = np.min(valuef)
        maxk = np.max(valuef)
        assert mink >= -abs(mValue), f"Problem with {grad}. Set a_min to -mValue in the np.clip call"
        assert maxk <= abs(mValue), f"Problem with {grad}.Set a_max to mValue in the np.clip call"
        index_not_clipped = np.logical_and(valuei <= mValue, valuei >= -mValue)
        assert np.all(valuei[index_not_clipped] == valuef[index_not_clipped]), f" Problem with {grad}. Some values that should not have changed, changed during the clipping process."
    
    print("\033[92mAll tests passed!\x1b[0m")
    
clip_test(clip, 10)
clip_test(clip, 5)
    


Gradients for mValue=10
gradients["dWaa"][1][2] = 10.0
gradients["dWax"][3][1] = -10.0
gradients["dWya"][1][2] = 0.2971381536101662
gradients["db"][4] = [10.]
gradients["dby"][1] = [8.45833407]
All tests passed!

Gradients for mValue=5
gradients["dWaa"][1][2] = 5.0
gradients["dWax"][3][1] = -5.0
gradients["dWya"][1][2] = 0.2971381536101662
gradients["db"][4] = [5.]
gradients["dby"][1] = [5.]
All tests passed!


**Expected values**
```
Gradients for mValue=10
gradients["dWaa"][1][2] = 10.0
gradients["dWax"][3][1] = -10.0
gradients["dWya"][1][2] = 0.2971381536101662
gradients["db"][4] = [10.]
gradients["dby"][1] = [8.45833407]

Gradients for mValue=5
gradients["dWaa"][1][2] = 5.0
gradients["dWax"][3][1] = -5.0
gradients["dWya"][1][2] = 0.2971381536101662
gradients["db"][4] = [5.]
gradients["dby"][1] = [5.]
```

### Sampling

After training the model, generating new text (characters) involves a sampling process, as illustrated below:

<img src="images/dinos3.png" style="width:500;height:300px;">
<caption><center><b>Figure 3</b>: Illustration of text generation using a trained model. Begin with 

$x^{\langle 1 \rangle} = \vec{0}$ at the first time step and sample one character at a time from the network.</center></caption>


### sample

The `sample` function are used to sample characters. 

There are 4 steps in `sample`:

- **Step 1**: Input the "dummy" vector of zeros $x^{\langle 1 \rangle} = \vec{0}$. 
    - This is the default input before generating any characters. 
      Also, we have, $a^{\langle 0 \rangle} = \vec{0}$

- **Step 2**: Run one step of forward propagation to get $a^{\langle 1 \rangle}$ and $\hat{y}^{\langle 1 \rangle}$. Here are the equations:

*hidden state:*  
$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t+1 \rangle } + W_{aa} a^{\langle t \rangle } + b)\tag{1}$$

*activation:*
$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y \tag{2}$$

*prediction:*
$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle })\tag{3}$$

- Details about $\hat{y}^{\langle t+1 \rangle }$:
   - Note that $\hat{y}^{\langle t+1 \rangle }$ is a (softmax) probability vector (its entries are between 0 and 1 and sum to 1). 
   - $\hat{y}^{\langle t+1 \rangle}_i$ represents the probability that the character indexed by "i" is the next character.  
   - A `softmax()` function is provided.

- **Step 3: Sampling**:
    - With $y^{\langle t+1 \rangle}$ available, the next task is to select the following letter in the sequence. To introduce variability and avoid generating the same result every time, use `np.random.choice` to select a letter based on its probability, rather than always picking the most probable one.
    - Choose the next character's **index** according to the probability distribution given by $\hat{y}^{\langle t+1 \rangle }$. 
    - For example, if $\hat{y}^{\langle t+1 \rangle }_i = 0.16$, the index "i" should be selected with a 16% probability.
    - Utilize [np.random.choice](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html) for this task.

    Here is an example of how to use `np.random.choice()`:
    ```python
    np.random.seed(0)
    probs = np.array([0.1, 0.0, 0.7, 0.2])
    idx = np.random.choice(range(len(probs)), p=probs)
    ```

    - This results in the index (`idx`) being chosen according to the distribution:
    
    $P(index = 0) = 0.1, P(index = 1) = 0.0, P(index = 2) = 0.7, P(index = 3) = 0.2$.

    - Ensure that the `p` parameter is a 1D vector of probabilities.
    - Note that $\hat{y}^{\langle t+1 \rangle}$, referred to as `y` in the code, is a 2D array.
    - While implementing, the first argument to `np.random.choice` should be an ordered list `[0, 1, ..., vocab_len-1]`. Avoid using `char_to_ix.values()` as the order of dictionary values might differ between runs, potentially affecting the results.

- **Step 4: Update $x^{\langle t \rangle }$**:
    - The final step in the `sample()` function is to update the variable `x`, which currently holds $x^{\langle t \rangle }$, with $x^{\langle t + 1 \rangle }$.
    - Represent $x^{\langle t + 1 \rangle }$ by creating a one-hot vector for the character selected as the prediction.
    - Forward propagate $x^{\langle t + 1 \rangle }$ in Step 1 of the loop and continue this process until a `"\n"` character is generated, signifying the end of the dinosaur name.


In [8]:
def sample(parameters, char_to_ix, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- Python dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix -- Python dictionary mapping each character to an index.
    seed -- Used for grading purposes. Do not worry about it.

    Returns:
    indices -- A list of length n containing the indices of the sampled characters.
    """
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    # Step 1: Create the a zero vector x that can be used as the one-hot vector 
    # Representing the first character (initializing the sequence generation).
    x = np.zeros((vocab_size,1))
    # Step 1': Initialize a_prev as zeros
    a_prev = np.zeros((n_a ,1))
    
    # Create an empty list of indices. This is the list which will contain the list of indices of the characters to generate
    indices = []
    
    # idx is the index of the one-hot vector x that is set to 1
    # All other positions in x are zero.
    # Initialize idx to -1
    idx = -1
    
    # Loop over time-steps t. At each time-step:
    # Sample a character from a probability distribution 
    # And append its index (`idx`) to the list "indices". 
    # Stop if it reach 50 characters 
    # (which should be very unlikely with a well-trained model).
    # Setting the maximum number of characters helps with debugging and prevents infinite loops. 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        a = np.tanh(np.dot(Wax,x) + np.dot(Waa,a_prev) + b)
        z = np.dot(Wya,a) + by
        y = softmax(z)
        
        # For grading purposes
        np.random.seed(counter + seed) 
        
        # Step 3: Sample the index of a character within the vocabulary from the probability distribution y
        # (see additional hints above)
        idx = np.random.choice(range(len(y)), p = np.squeeze(y) )

        # Append the index to "indices"
        indices.append(idx)
        
        # Step 4: Overwrite the input x with one that corresponds to the sampled index `idx`.
        # (see additional hints above)
        x = np.zeros((vocab_size,1))
        x[idx] = 1
        
        # Update "a_prev" to be "a"
        a_prev = a
        
        # for grading purposes
        seed += 1
        counter += 1

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [9]:
def sample_test(target):
    np.random.seed(24)
    _, n_a = 20, 100
    Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
    b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}


    indices = target(parameters, char_to_ix, 0)
    print("Sampling:")
    print("list of sampled indices:\n", indices)
    print("list of sampled characters:\n", [ix_to_char[i] for i in indices])
    
    assert len(indices) < 52, "Indices length must be smaller than 52"
    assert indices[-1] == char_to_ix['\n'], "All samples must end with \\n"
    assert min(indices) >= 0 and max(indices) < len(char_to_ix), f"Sampled indexes must be between 0 and len(char_to_ix)={len(char_to_ix)}"
    assert np.allclose(indices[0:6], [23, 16, 26, 26, 24, 3]), "Wrong values"
    
    print("\033[92mAll tests passed!")

sample_test(sample)

Sampling:
list of sampled indices:
 [23, 16, 26, 26, 24, 3, 21, 1, 7, 24, 15, 3, 25, 20, 6, 13, 10, 8, 20, 12, 2, 0]
list of sampled characters:
 ['w', 'p', 'z', 'z', 'x', 'c', 'u', 'a', 'g', 'x', 'o', 'c', 'y', 't', 'f', 'm', 'j', 'h', 't', 'l', 'b', '\n']
All tests passed!


**Expected output**
```
Sampling:
list of sampled indices:
 [23, 16, 26, 26, 24, 3, 21, 1, 7, 24, 15, 3, 25, 20, 6, 13, 10, 8, 20, 12, 2, 0]
list of sampled characters:
 ['w', 'p', 'z', 'z', 'x', 'c', 'u', 'a', 'g', 'x', 'o', 'c', 'y', 't', 'f', 'm', 'j', 'h', 't', 'l', 'b', '\n']
```

#### Key Points

* Exploding gradients occur when updates are excessively large, causing them to overshoot the optimal values during backpropagation, which complicates training.
    * **Solution:** Clip gradients before updating the parameters to prevent this issue.

* Sampling is a technique used to select the index of the next character based on a probability distribution.
    * **Steps for character-level sampling:**
        * Initialize with a "dummy" vector of zeros as the starting input.
        * Perform one step of forward propagation to obtain $a^{\langle 1 \rangle}$ (the first character) and $\hat{y}^{\langle 1 \rangle}$ (the probability distribution for the next character).
        * Use `np.random.choice` to sample from the probability distribution, which introduces variability and prevents repetitive results, making the generated names more interesting.


## Building the Language Model

It's time to build the character-level language model for text generation.

### Gradient Descent

In this section, a function performing one step of stochastic gradient descent with gradient clipping will be implemented. The process involves iterating through the training examples one at a time, applying stochastic gradient descent for optimization.

#### Optimization Loop for an RNN

The typical optimization loop for an RNN includes the following steps:

1. **Forward Propagation:** 
   - Compute the loss by propagating through the RNN.

2. **Backward Propagation Through Time:**
   - Calculate the gradients of the loss with respect to the parameters.

3. **Gradient Clipping:**
   - Clip the gradients to prevent issues with exploding gradients.

4. **Parameter Update:**
   - Update the parameters using gradient descent based on the clipped gradients.


### optimize

This function implements the optimization process (one step of stochastic gradient descent).

The following functions are provided in `utils.py`:
`rnn_forward`, `rnn_backward` and `update_parameters`.

<!-- ```python
def rnn_forward(X, Y, a_prev, parameters):
    """ Performs the forward propagation through the RNN and computes the cross-entropy loss.
    It returns the loss' value as well as a "cache" storing values to be used in backpropagation."""
    ....
    return loss, cache
    
def rnn_backward(X, Y, parameters, cache):
    """ Performs the backward propagation through time to compute the gradients of the loss with respect
    to the parameters. It returns also all the hidden states."""
    ...
    return gradients, a

def update_parameters(parameters, gradients, learning_rate):
    """ Updates parameters using the Gradient Descent Update Rule."""
    ...
    return parameters
```

Recall that you previously implemented the `clip` function: -->

#### Parameters

- The `parameters` dictionary, which contains the weights and biases, is updated during optimization. Although `parameters` is not one of the returned values from the `optimize` function, changes made to it within the function persist outside of it. This is because `parameters` is passed by reference.
- In Python, dictionaries and lists are passed by reference. Thus, if a dictionary or list is passed to a function and modified within that function, the changes are applied to the original dictionary or list, not a copy.


In [10]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    Execute one step of the optimization to train the model.
    
    Arguments:
    X -- list of integers, where each integer is a number that maps to a character in the vocabulary.
    Y -- list of integers, exactly the same as X but shifted one index to the left.
    a_prev -- previous hidden state.
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    learning_rate -- learning rate for the model.
    
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- python dictionary containing:
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
                        db -- Gradients of bias vector, of shape (n_a, 1)
                        dby -- Gradients of output bias vector, of shape (n_y, 1)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    """
    # Read the descriptions for the following functions in utils.py
    # Forward propagate through time
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # Backpropagate through time
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    # Clip your gradients between -5 (min) and 5 (max)
    gradients = clip(gradients, 5)
    
    # Update parameters
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    return loss, gradients, a[len(X)-1]

In [11]:
def optimize_test(target):
    np.random.seed(1)
    vocab_size, n_a = 27, 100
    a_prev = np.random.randn(n_a, 1)
    Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
    b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}
    X = [12, 3, 5, 11, 22, 3]
    Y = [4, 14, 11, 22, 25, 26]
    old_parameters = copy.deepcopy(parameters)
    loss, gradients, a_last = target(X, Y, a_prev, parameters, learning_rate = 0.01)
    print("Loss =", loss)
    print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
    print("np.argmax(gradients[\"dWax\"]) =", np.argmax(gradients["dWax"]))
    print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
    print("gradients[\"db\"][4] =", gradients["db"][4])
    print("gradients[\"dby\"][1] =", gradients["dby"][1])
    print("a_last[4] =", a_last[4])
    
    assert np.isclose(loss, 126.5039757), "Problems with the call of the rnn_forward function"
    for grad in gradients.values():
        assert np.min(grad) >= -5, "Problems in the clip function call"
        assert np.max(grad) <= 5, "Problems in the clip function call"
    assert np.allclose(gradients['dWaa'][1, 2], 0.1947093), "Unexpected gradients. Check the rnn_backward call"
    assert np.allclose(gradients['dWya'][1, 2], -0.007773876), "Unexpected gradients. Check the rnn_backward call"
    assert not np.allclose(parameters['Wya'], old_parameters['Wya']), "parameters were not updated"
    
    print("\033[92mAll tests passed!")

optimize_test(optimize)

Loss = 126.50397572165389
gradients["dWaa"][1][2] = 0.19470931534713587
np.argmax(gradients["dWax"]) = 93
gradients["dWya"][1][2] = -0.007773876032002162
gradients["db"][4] = [-0.06809825]
gradients["dby"][1] = [0.01538192]
a_last[4] = [-1.]
All tests passed!


**Expected output**

```
Loss = 126.50397572165389
gradients["dWaa"][1][2] = 0.19470931534713587
np.argmax(gradients["dWax"]) = 93
gradients["dWya"][1][2] = -0.007773876032002162
gradients["db"][4] = [-0.06809825]
gradients["dby"][1] = [0.01538192]
a_last[4] = [-1.]
```

### Training the Model 

* Given the dataset of dinosaur names, you'll use each line of the dataset (one name) as one training example. 
* Every 2000 steps of stochastic gradient descent, you will sample several randomly chosen names to see how the algorithm is doing. 
 
### model

When `examples[index]` contains one dinosaur name (string), to create an example (X, Y), we can use this:

##### Set the index `idx` into the list of examples

- Iterate through the shuffled list of dinosaur names in the "examples" list using a for-loop.
- If there are `n_e` examples and the index exceeds `n_e`, the index should cycle back to 0. This ensures continuous feeding of examples into the model when `j` is `n_e`, `n_e + 1`, etc.
- Hint: `(n_e + 1) % n_e` equals 1, which represents the remainder when `(n_e + 1)` is divided by `n_e`.
- `%` is the [modulo operator in Python](https://www.freecodecamp.org/news/the-python-modulo-operator-what-does-the-symbol-mean-in-python-solved/).

##### Extract a single example from the list of examples
- Use the previously set `idx` index to retrieve one word from the "examples" list, referred to as `single_example`.


##### Convert a string into a list of characters: `single_example_chars`
- `single_example_chars`: A string is a list of characters.
- Use a list comprehension (recommended over for-loops) to generate a list of characters.

```Python
str = 'I love deep learning'
list_of_chars = [c for c in str]
print(list_of_chars)
```

```
['I', ' ', 'l', 'o', 'v', 'e', ' ', 'l', 'e', 'a', 'r', 'n', 'i', 'n', 'g']
```

* For more on [list comprehensions](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions):

##### Convert list of characters to a list of integers: `single_example_ix`
* Create a list that contains the index numbers associated with each character.
* Use the dictionary `char_to_ix`
* Combine this with the list comprehension that is used to get a list of characters from a string.

##### Create the list of input characters: `X`
* `rnn_forward` uses the **`None`** value as a flag to set the input vector as a zero-vector.
* Prepend the list [**`None`**] in front of the list of input character *indices*.
* There is more than one way to prepend a value to a list.  One way is to add two lists together: `['a'] + ['b']`

##### Get the integer representation of the newline character `ix_newline`
* `ix_newline`: The newline character signals the end of the dinosaur name.
    - Get the integer representation of the newline character `'\n'`.
    - Use `char_to_ix`

##### Set the list of labels (integer representation of the characters): `Y`
* The goal is to train the RNN to predict the next letter in the name, so the labels are the list of characters that are one time-step ahead of the characters in the input `X`.
    - For example, `Y[0]` contains the same value as `X[1]`  
* The RNN should predict a newline at the last letter, so add `ix_newline` to the end of the labels. 
    - Append the integer representation of the newline character to the end of `Y`.
    - Note that `append` is an in-place operation.
    - It might be easier for you to add two lists together.

In [12]:
def model(data_x, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27, verbose = False):
    """
    Trains the model and generates dinosaur names. 
    
    Arguments:
    data_x -- text corpus, divided in words
    ix_to_char -- dictionary that maps the index to a character
    char_to_ix -- dictionary that maps a character to an index
    num_iterations -- number of iterations to train the model for
    n_a -- number of units of the RNN cell
    dino_names -- number of dinosaur names you want to sample at each iteration. 
    vocab_size -- number of unique characters found in the text (size of the vocabulary)
    
    Returns:
    parameters -- learned parameters
    """
    
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = vocab_size, vocab_size
    
    # Initialize parameters
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # Initialize loss (this is required because we want to smooth our loss)
    loss = get_initial_loss(vocab_size, dino_names)
    
    # Build list of all dinosaur names (training examples).
    examples = [x.strip() for x in data_x]
    
    # Shuffle list of all dinosaur names
    np.random.seed(0)
    np.random.shuffle(examples)
    
    # Initialize the hidden state of your LSTM
    a_prev = np.zeros((n_a, 1))
    
    # for grading purposes
    last_dino_name = "abc"
    
    # Optimization loop
    for j in range(num_iterations):
        
        # Set the index `idx` (see instructions above)
        idx = j % len(examples)
        
        # Set the input X (see instructions above)
        single_example_chars = examples[idx]
        single_example_ix = [char_to_ix[c] for c in single_example_chars]
        X = [None] + single_example_ix
        
        # Set the labels Y (see instructions above)
        ix_newline = [char_to_ix["\n"]]
        Y = X[1:] + ix_newline

        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
        
        # debug statements to aid in correctly forming X, Y
        if verbose and j in [0, len(examples) -1, len(examples)]:
            print("j = " , j, "idx = ", idx,) 
        if verbose and j in [0]:
#             print("single_example =", single_example)
            print("single_example_chars", single_example_chars)
            print("single_example_ix", single_example_ix)
            print(" X = ", X, "\n", "Y =       ", Y, "\n")
        
        # to keep the loss smooth.
        loss = smooth(loss, curr_loss)

        # Every 2000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            # The number of dinosaur names to print
            seed = 0
            for name in range(dino_names):
                
                # Sample indices and print them
                sampled_indices = sample(parameters, char_to_ix, seed)
                last_dino_name = get_sample(sampled_indices, ix_to_char)
                print(last_dino_name.replace('\n', ''))
                
                seed += 1  # To get the same result (for grading purposes), increment the seed by one. 
      
            print('\n')
        
    return parameters, last_dino_name

When you run the following cell, you should observe your model outputting random-looking characters at the first iteration. After a few thousand iterations, your model should learn to generate reasonable-looking names. 

In [13]:
parameters, last_name = model(data.split("\n"), ix_to_char, char_to_ix, 22001, verbose = True)

assert last_name == 'Trodonosaurus\n', "Wrong expected output"
print("\033[92mAll tests passed!")

j =  0 idx =  0
single_example_chars turiasaurus
single_example_ix [20, 21, 18, 9, 1, 19, 1, 21, 18, 21, 19]
 X =  [None, 20, 21, 18, 9, 1, 19, 1, 21, 18, 21, 19] 
 Y =        [20, 21, 18, 9, 1, 19, 1, 21, 18, 21, 19, 0] 

Iteration: 0, Loss: 23.087336

Nkzxwtdmfqoeyhsqwasjkjvu
Kneb
Kzxwtdmfqoeyhsqwasjkjvu
Neb
Zxwtdmfqoeyhsqwasjkjvu
Eb
Xwtdmfqoeyhsqwasjkjvu


j =  1535 idx =  1535
j =  1536 idx =  0
Iteration: 2000, Loss: 27.884160

Liusskeomnolxeros
Hmdaairus
Hytroligoraurus
Lecalosapaus
Xusicikoraurus
Abalpsamantisaurus
Tpraneronxeros


Iteration: 4000, Loss: 25.901815

Mivrosaurus
Inee
Ivtroplisaurus
Mbaaisaurus
Wusichisaurus
Cabaselachus
Toraperlethosdarenitochusthiamamumamaon


Iteration: 6000, Loss: 24.608779

Onwusceomosaurus
Lieeaerosaurus
Lxussaurus
Oma
Xusteonosaurus
Eeahosaurus
Toreonosaurus


Iteration: 8000, Loss: 24.070350

Onxusichepriuon
Kilabersaurus
Lutrodon
Omaaerosaurus
Xutrcheps
Edaksoje
Trodiktonus


Iteration: 10000, Loss: 23.844446

Onyusaurus
Klecalosaurus
Lust

**Expected output**
```
...
Iteration: 22000, Loss: 22.728886

Onustreofkelus
Llecagosaurus
Mystolojmiaterltasaurus
Ola
Yuskeolongus
Eiacosaurus
Trodonosaurus
```

### Conclusion

The algorithm has started to generate plausible dinosaur names towards the end of training. Initially, it produced random characters, but over time, recognizable dinosaur names with interesting endings emerged. Running the algorithm longer and adjusting hyperparameters might yield even better results! Names like `maconucon`, `marloralus`, and `macingsersaurus` were generated, indicating that the model has learned patterns such as dinosaur names ending in `saurus`, `don`, `aura`, `tor`, etc.

If some generated names are not cool, it's worth noting that not all real dinosaur names are either (e.g., `dromaeosauroides` is a real name from the training set). The model provides a set of candidates, allowing selection of the coolest names.

 A small dataset is used for quick training on a CPU. Training a model for the English language requires a larger dataset, more computation, and longer training times on GPUs. Despite the brief training, names like the great and fierce **Mangosaurus** were among the favorites!

<img src="images/mangosaurus.jpeg" style="width:250;height:300px;">


## Writing like Shakespeare

A similar task to character-level text generation (but more complex) is generating Shakespearean poems. Instead of learning from a dataset of dinosaur names, a collection of Shakespearean poems can be used. With LSTM cells, longer-term dependencies spanning many characters in the text can be learned—for example, a character appearing at one point in a sequence can influence what appears much later. These long-term dependencies were less critical with dinosaur names, as the names are relatively short.

<img src="images/shakespeare.jpg" style="width:500;height:400px;">
<caption><center><b>Let's become poets!</b></center></caption>

Below, a Shakespeare poem generator with Keras can be implemented. Run the following cell to load the required packages and models. This may take a few minutes.


In [14]:
from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import get_file
from tensorflow.keras.preprocessing.sequence import pad_sequences
from shakespeare_utils import *
import sys
import io

Loading text data...
Creating training set...
number of training examples: 31412
Vectorizing training set...
Loading model...


To save you some time, a model has already been trained for ~1000 epochs on a collection of Shakespearean poems called "<i>[The Sonnets](shakespeare.txt)</i>."

Train the model for one more epoch. After the training completes (this will take a few minutes), run `generate_output`. This will prompt for an input (less than 40 characters). The poem will start with the provided sentence, and the RNN Shakespeare will complete the rest of the poem. For example, try "Forsooth this maketh no sense" (without the quotation marks). The results may vary depending on whether a space is included at the end, so experiment with both options and try other inputs as well.

In [15]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, batch_size=128, epochs=1, callbacks=[print_callback])

246/246 [==============================] - 123s 501ms/step - loss: 2.5729


In [16]:
# Run this cell to try with different inputs without having to re-train the model 
generate_output()

Write the beginning of your poem, the Shakespeare machine will complete it. Your input is: The model 


Here is your poem: 

The model mins ans ele suland:
to praviout unlath or to mr fill fer,
why cofdol's my evine sue best even thy poy,
and my betanch worls dither bame enredces,
on thises wo should of lend ally heaor,
with a credy as i conlest her will new.
thy mower doth to dithath shume's levery?
bet ctronga enleanes that is new she that,
whone eyes and well veose thicht chlight,
when is a were best autthrrongs yew wascuts cr

The RNN Shakespeare model is very similar to the one you built for dinosaur names. The only major differences are:
- LSTMs instead of the basic RNN to capture longer-range dependencies
- The model is a deeper, stacked LSTM model (2 layer)
- Using Keras instead of Python to simplify the code 


<a name='5'></a>
## 5 - References 
- This exercise took inspiration from Andrej Karpathy's implementation: https://gist.github.com/karpathy/d4dee566867f8291f086. To learn more about text generation, also check out Karpathy's [blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).